In [42]:
import numpy as np
from ete3 import Tree
import pandas as pd

In [43]:
#Distance matrix

def read_mtrx(filename):
    f = open(filename,'r')
    num_species = int(f.readline())

    dict_mat = {}
    mat = np.zeros((num_species,num_species))
    
    for i in range(0,num_species):
        line = f.readline()
        nums_in_line = line.split()
        dict_mat[nums_in_line[0]] = i
        for j in range(1,num_species +1):
            mat[i,j-1] = nums_in_line[j]
    f.close()
    return dict_mat, mat 

In [44]:
species_dict, dist = read_mtrx("C:\\Users\\awila\\Downloads\\example_slide4.phy")
print(species_dict)
print(dist)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
[[0.   0.23 0.16 0.2  0.17]
 [0.23 0.   0.23 0.17 0.24]
 [0.16 0.23 0.   0.2  0.11]
 [0.2  0.17 0.2  0.   0.21]
 [0.17 0.24 0.11 0.21 0.  ]]


In [45]:
species = list(species_dict.keys())
species

['A', 'B', 'C', 'D', 'E']

In [46]:
list(species_dict.keys())

['A', 'B', 'C', 'D', 'E']

In [47]:
dict_dist= {}
for i in range(len(species)) :
    dict_dist[species[i]] = dist[i]
dict_dist

{'A': array([0.  , 0.23, 0.16, 0.2 , 0.17]),
 'B': array([0.23, 0.  , 0.23, 0.17, 0.24]),
 'C': array([0.16, 0.23, 0.  , 0.2 , 0.11]),
 'D': array([0.2 , 0.17, 0.2 , 0.  , 0.21]),
 'E': array([0.17, 0.24, 0.11, 0.21, 0.  ])}

In [48]:
df=pd.DataFrame.from_dict(dict_dist,
orient='index', columns=species)
print(df)

      A     B     C     D     E
A  0.00  0.23  0.16  0.20  0.17
B  0.23  0.00  0.23  0.17  0.24
C  0.16  0.23  0.00  0.20  0.11
D  0.20  0.17  0.20  0.00  0.21
E  0.17  0.24  0.11  0.21  0.00


In [49]:
df.drop("A")

,A,B,C,D,E
B,0.23,0.00,0.23,0.17,0.24
C,0.16,0.23,0.00,0.20,0.11
D,0.20,0.17,0.20,0.00,0.21
E,0.17,0.24,0.11,0.21,0.00


In [79]:
df.loc['D','B']

0.17

In [80]:
df.size

25

In [50]:
#Initialization of starlike tree

ntree = "("
species = list(species_dict.keys())
for i in range(len(species)-1):
    ntree += str(species[i]) + ","
ntree += str(species[len(species)-1])+")"
print(ntree)

(A,B,C,D,E)


In [51]:
t = Tree("(A,B,C,D,E);", format=1)
print(t)


   /-A
  |
  |--B
  |
--|--C
  |
  |--D
  |
   \-E


In [52]:
from Bio import Phylo
from io import StringIO
tree = Phylo.read(StringIO("(A, B, C, D, E)"), "newick")
print(tree)

Tree(rooted=False, weight=1.0)
    Clade()
        Clade(name='A')
        Clade(name='B')
        Clade(name='C')
        Clade(name='D')
        Clade(name='E')


In [53]:
t1 = Phylo.read(StringIO(ntree), "newick")
print(t1)

Tree(rooted=False, weight=1.0)
    Clade()
        Clade(name='A')
        Clade(name='B')
        Clade(name='C')
        Clade(name='D')
        Clade(name='E')


In [54]:
#NJ algorithm Step 1: finding the two neighbours

dist.shape[0]

5

In [55]:
new_N = np.full([5,5], None)
new_N

array([[None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None],
       [None, None, None, None, None]], dtype=object)

In [65]:
ris = []
for i in range(5):
    ri = 0
    for j in range(5):
        ri +=dist[i,j]
    ris.append(ri/3)
ris

[0.25333333333333335, 0.29, 0.23333333333333336, 0.26, 0.24333333333333332]

In [66]:
rjs = []
for j in range(5):
    rj = 0
    for i in range(5):
        rj +=dist[i,j]
    rjs.append(rj/3)
rjs

[0.25333333333333335, 0.29, 0.23333333333333336, 0.26, 0.24333333333333332]

In [67]:
minN = dist[0,0]+ris[0]+rjs[0]
mini = 0
minj = 0
for i in range(5):
    for j in range(5):
        nij = dist[i,j]+ris[i]+rjs[j]
        new_N[i,j]=nij
        if nij<minN : 
            minN = nij
            mini = i
            minj = j
print(new_N)
print(mini,minj,minN)

[[0.5066666666666667 0.7733333333333334 0.6466666666666667
  0.7133333333333334 0.6666666666666666]
 [0.7733333333333334 0.58 0.7533333333333334 0.72 0.7733333333333333]
 [0.6466666666666667 0.7533333333333334 0.46666666666666673
  0.6933333333333334 0.5866666666666667]
 [0.7133333333333334 0.72 0.6933333333333334 0.52 0.7133333333333333]
 [0.6666666666666667 0.7733333333333332 0.5866666666666667
  0.7133333333333334 0.48666666666666664]]
2 2 0.46666666666666673


In [68]:
def step1(dist):
    S = dist.shape[0]
    new_N = np.full([S,S], None)
    ris = []
    for i in range(S):
        ri = 0
        for j in range(S):
            ri +=dist[i,j]
        ris.append(ri/(S-2))
    
    rjs = []
    for j in range(5):
        rj = 0
        for i in range(5):
            rj +=dist[i,j]
        rjs.append(rj/3)
        
    minN = dist[0,0]+ris[0]+rjs[0]
    mini = 0
    minj = 0
    for i in range(5):
        for j in range(5):
            nij = dist[i,j]+ris[i]+rjs[j]
            new_N[i,j]=nij
            if nij<minN : 
                minN = nij
                mini = i
                minj = j
    return(mini,minj,minN)

In [69]:
step1(dist)

(2, 2, 0.46666666666666673)

In [70]:
#Step 2: Adding a new node k

species_dict, dist = read_mtrx("C:\\Users\\awila\\Downloads\\example_slide4.phy")
print(species_dict)
print(dist)

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
[[0.   0.23 0.16 0.2  0.17]
 [0.23 0.   0.23 0.17 0.24]
 [0.16 0.23 0.   0.2  0.11]
 [0.2  0.17 0.2  0.   0.21]
 [0.17 0.24 0.11 0.21 0.  ]]


In [71]:
species = list(species_dict.keys())
species

['A', 'B', 'C', 'D', 'E']

In [73]:
node_list = list(species_dict.keys())
node_list

['A', 'B', 'C', 'D', 'E']

In [74]:
node_list = list(species_dict.keys())
node_list.remove(species[mini])
node_list.remove(species[minj])
node_list.append('('+species[mini]+','+species[minj]+')')
node_list

ValueError: list.remove(x): x not in list

In [41]:
def step2(mini,minj,node_list):
    nodes = node_list.copy()
    node_list.remove(nodes[mini])
    node_list.remove(nodes[minj])
    node_list.append('('+nodes[mini]+','+nodes[minj]+')')
    return node_list

In [75]:
step2(mini,minj,node_list)

ValueError: list.remove(x): x not in list